In [22]:
import yfinance as yf
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd
from datetime import datetime

# === PARAMETERS ===
ticker = "^NSEI"
portfolio_value = 1e7  # ₹1 Cr
lot_size = 50
beta_map = {'Bullish': 1.5, 'Neutral': 0.75, 'Bearish': 0.5}

# === FETCH DAILY DATA (recent 6 months) ===
data = yf.download(ticker, period="6mo", interval="1d")
if data.empty:
    raise ValueError("No data fetched. Check ticker or internet connection.")

# Determine price column
price_col = 'Close' if 'Close' in data.columns else 'Adj Close'
data = data[[price_col]].copy()
data.rename(columns={price_col: 'Price'}, inplace=True)

# === CHECK IF TODAY IS MONTH-END ===
today = data.index[-1].date()  # last available trading day
last_day_of_month = (today + MonthEnd(0)).date()

if today != last_day_of_month:
    print(f"⚠️ Warning: Today ({today}) is not month-end ({last_day_of_month}).")
    print("Calculation will still run using the latest available price.\n")

# === CALCULATE 200DMA AND 3-MONTH SLOPE ===
data['200DMA'] = data['Price'].rolling(window=200, min_periods=1).mean()
data['DMA_slope'] = data['200DMA'].diff(63)  # ~3 months slope

latest = data.iloc[-1]
spot = latest['Price'].item()
dma_200 = latest['200DMA'].item()
slope = latest['DMA_slope'].item()
as_of = latest.name.date()

# === DETERMINE TREND ===
if spot > dma_200 and slope > 0:
    trend = 'Bullish'
elif spot > dma_200:
    trend = 'Neutral'
else:
    trend = 'Bearish'

beta = beta_map[trend]

# === FUTURES TARGET ===
futures_notional = (beta - 1) * portfolio_value
lots = int(round(futures_notional / (spot * lot_size)))

# === DISPLAY TABLE ===
summary = pd.DataFrame({
    "Date": [as_of],
    "Nifty Spot": [round(spot,2)],
    "200DMA": [round(dma_200,2)],
    "3M Slope": [round(slope,2)],
    "Trend": [trend],
    "Beta (Overlay)": [beta],
    "Portfolio (₹ Cr)": [portfolio_value/1e7],
    "Futures Notional (₹)": [round(futures_notional,2)],
    "Lots to Hold": [abs(lots)],
    "Position": ["LONG" if lots > 0 else "SHORT" if lots < 0 else "FLAT"]
})

print("=== Monthly Overlay Strategy Signal ===")
display(summary)


/var/folders/59/27lpl20d01l3sj6sf_qj_1300000gn/T/ipykernel_72743/423793900.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="6mo", interval="1d")
[*********************100%***********************]  1 of 1 completed

⚠️ Warning: Today (2025-08-26) is not month-end (2025-08-31).
Calculation will still run using the latest available price.

=== Monthly Overlay Strategy Signal ===


,Date,Nifty Spot,200DMA,3M Slope,Trend,Beta (Overlay),Portfolio (₹ Cr),Futures Notional (₹),Lots to Hold,Position
0,2025-08-26,24712.05,24330.79,693.98,Bullish,1.5,1.0,5000000.0,4,LONG
